# Oļegs Korsaks, 051RDB146

In [1]:
from math import exp
from operator import methodcaller

import numpy as np
import tensorflow as tf

from sklearn.datasets import load_iris

In [2]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1]],
]

In [3]:
def a_step(val):
    return int(val>0)


def a_lin(val):
    if val <= 0:
        return 0
    elif val > 5:
        return 1
    
    return val/5


def a_sigmoid(val):
    return 1.0 / (1.0+exp(-val))


def a_tanh(val):
    return (exp(val)-exp(-val)) / (exp(val)+exp(-val))

In [4]:
learning_rate = 0.1


class Neiron:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_step(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [5]:
def run_neirons(data: list, neiron_class, in_size: int, class_count: int, epoch_count: int = 100):
    neirons = tuple(neiron_class(in_size) for _ in range(class_count))
    
    for epoch in range(epoch_count):
        for sample in data:
            for i, neiron in enumerate(neirons):
                neiron.learn(sample[1], sample[2][i])
    
    for sample in data:
        method = methodcaller('classify', sample[1])
        print(f'{sample[0]} {tuple(map(method, neirons))}')
    
    for neiron in neirons:
        print(neiron.weights)

In [6]:
run_neirons(letters, Neiron, 15, 3, 100)

A (1, 0, 0)
E (1, 0, 0)
D (1, 0, 0)
B (0, 1, 0)
C (0, 1, 0)
F (0, 1, 0)
G (0, 0, 1)
H (0, 0, 1)
I (0, 0, 1)
[-0.7  0.5 -0.8  0.  -0.6 -0.3  0.   0.9  1.8  0.  -0.6 -1.   0.   0.3
  0.4]
[ 0.2  0.2 -0.4  0.2 -0.2  0.4  0.2 -0.7 -1.7  0.2 -0.2 -0.1  0.2  0.2
 -0.4]
[ 0.2 -0.3  0.3 -0.2  0.3 -0.1 -0.2 -0.2  0.4 -0.2  0.3  0.4 -0.2  0.1
 -0.1]


In [7]:
iris = load_iris()
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(4, )))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')

targets = np.zeros((len(iris.target), 3))

for ti, iris_target in enumerate(iris.target):
    targets[ti][iris_target] = 1

model.fit(iris.data, targets, epochs=1000, verbose=0)
predictions = model.predict(iris.data)

print(predictions)

[[0.9367028  0.05823039 0.00506684]
 [0.9321774  0.06218387 0.00563866]
 [0.94337815 0.05235055 0.00427131]
 [0.9314676  0.06279203 0.00574044]
 [0.93935883 0.05589602 0.00474522]
 [0.91006255 0.08121295 0.00872453]
 [0.9390459  0.05616045 0.00479361]
 [0.93028617 0.06382884 0.00588492]
 [0.9375305  0.05749181 0.00497769]
 [0.93020076 0.06390585 0.00589334]
 [0.92908293 0.06488377 0.00603324]
 [0.92643595 0.0671621  0.006402  ]
 [0.93646383 0.05843861 0.00509763]
 [0.9566551  0.04051626 0.00282872]
 [0.9457105  0.05030462 0.00398498]
 [0.9293473  0.06465074 0.00600198]
 [0.93871313 0.05646665 0.00482023]
 [0.9338325  0.06073857 0.00542892]
 [0.9062929  0.08443095 0.00927623]
 [0.931903   0.06241813 0.00567887]
 [0.90802723 0.08295728 0.00901556]
 [0.927192   0.06650852 0.00629949]
 [0.961385   0.03624966 0.0023653 ]
 [0.898906   0.09062811 0.01046592]
 [0.90171075 0.0882649  0.01002435]
 [0.9160657  0.07609773 0.00783661]
 [0.91605234 0.07609625 0.00785135]
 [0.9288998  0.06503863 0.00

## Mājasdarbs 1

Pārveidosim neirona klasi, lai viņš varētu klasificēt 4 klašu simbolus. Pievienosim ! _ un . simbolus

In [8]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1,0]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1,0]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1,0]],
    
    ['!',[0,1,0,0,1,0,0,1,0,0,0,0,0,1,0],[0,0,0,1]],
    ['_',[0,0,0,0,0,0,0,0,0,0,0,0,1,1,1],[0,0,0,1]],
    ['.',[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,1]],
]


class NeironHome:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_sigmoid(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [9]:
run_neirons(letters, NeironHome, 15, 4, 1000)

A (0.9975535051097201, 6.660629571448497e-06, 0.008248493415553408, 0.00013232113048282796)
E (0.8293688905071558, 0.07107663617372244, 0.00345475745807826, 0.0008908342725369956)
D (0.8834289329308431, 0.040197189905192325, 0.005393751338105291, 0.000374360388021805)
B (0.06296538404255056, 0.9611077227614221, 2.5487150667717718e-05, 0.00044414160389372084)
C (0.08063124850588929, 0.941473518306242, 0.00588321544731504, 0.0016078235509806357)
F (0.02633476838322883, 0.9911138221082045, 0.0034084107149066696, 2.9723027415453927e-05)
G (0.04066958299013252, 0.00015968439826207983, 0.9849054420714136, 0.0002711874602893066)
H (0.02117992296299061, 1.3000067100018376e-05, 0.9923537292619337, 1.8435006117880236e-05)
I (0.0037253089873815924, 2.5566882336613036e-05, 0.9800763983367607, 0.02047026562186848)
! (0.018415486080804077, 4.3871158090628676e-05, 0.013755496597630701, 0.9781077632338467)
_ (0.00838609151522248, 0.00038552142987693667, 0.0020254630608827185, 0.9968259609187382)
. (0.

**Ir redzams, ka "!", "\_" un "." simboli tiek klasificēti kā 4. klase. Citu burtu klasifikācija nemainās.**

Pārtaisīsim TensorFlow kodu, lai viņš klasificētu burtus:

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(15, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(15, )))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')

np_letters = np.array(tuple(np.array(letter[1]) for letter in letters))
print(np_letters)
targets = np.array(tuple(np.array(letter[2]) for letter in letters))
print(targets)

model.fit(np_letters, targets, epochs=1000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[0 1 0 1 0 1 1 1 1 1 0 1 1 0 1]
 [1 1 1 1 0 0 1 1 0 1 0 0 1 1 1]
 [1 1 0 1 0 1 1 0 1 1 0 1 1 1 0]
 [1 1 0 1 0 1 1 1 0 1 0 1 1 1 0]
 [1 1 1 1 0 0 1 0 0 1 0 0 1 1 1]
 [1 1 1 1 0 0 1 1 0 1 0 0 1 0 0]
 [1 1 1 1 0 0 1 0 1 1 0 1 1 1 1]
 [1 0 1 1 0 1 1 1 1 1 0 1 1 0 1]
 [0 1 0 0 1 0 0 1 0 0 1 0 0 1 0]
 [0 1 0 0 1 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
[[0.27332932 0.2654332  0.25105006 0.2101874 ]
 [0.2647939  0.2694666  0.24997689 0.21576257]
 [0.2722517  0.26801807 0.24935403 0.21037628]
 [0.27126005 0.27120554 0.24743117 0.21010324]
 [0.2618771  0.26605403 0.2495924  0.22247644]
 [0.26596656 0.27264377 0.24899675 0.21239291]
 [0.27034226 0.2694362  0.25340682 0.2068147 ]
 [0.27491257 0.27235618 0.25443473 0.19829652]
 [0.21114476 0.2123964  0.24763891 0.32881993]
 [0.21181244 0.21245801 0.24528062 0.33044887]


**Interesanti**, bet TensorFlow versija apstiprināja manas šaubas par "I" un "!" lidzīguma problēmu, kāmēr bez TensorFlow klasifikācija nostrādāja pareizi. Ar parējiem burtiem arī ir redzamas problēmas - B tiek pieskaitīts pie 2. klases. Pārbaudīšu, vai epohu skaits uzlabos situāciju:

In [13]:
model.fit(np_letters, targets, epochs=10000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[9.97870922e-01 1.15838971e-04 1.90411939e-03 1.09104018e-04]
 [8.84839237e-01 1.01547353e-01 1.31737711e-02 4.39652416e-04]
 [9.58204806e-01 3.25486399e-02 8.80875252e-03 4.37759649e-04]
 [2.94383299e-02 9.69883859e-01 1.31538123e-04 5.46289841e-04]
 [3.39466780e-02 9.64004815e-01 1.56627712e-03 4.82150848e-04]
 [1.68621377e-03 9.98041630e-01 1.52749650e-04 1.19389457e-04]
 [1.20214690e-02 1.79684634e-04 9.87736583e-01 6.23314772e-05]
 [2.32085376e-03 4.00965437e-05 9.97522056e-01 1.17011608e-04]
 [1.44508958e-03 2.53505768e-05 9.91538048e-01 6.99146651e-03]
 [1.04670261e-03 1.32174420e-04 4.31190850e-03 9.94509280e-01]
 [6.25429457e-05 1.08985434e-04 1.62381315e-04 9.99666095e-01]
 [1.33661160e-06 7.01762838e-05 5.75142039e-05 9.99870896e-01]]
